# Export TFLITE

In [1]:
import os
import tensorflow as tf
from ultralytics import YOLO
from onnx2tf import convert

# Paths
PT_MODEL_PATH = 'models/focus1/Focus1_YOLO11s_x1024_14112024.pt'
ONNX_MODEL_PATH = 'models/focus1/Focus1_YOLO11s_x1024_14112024.onnx'
TF_SAVED_MODEL_DIR = 'models/focus1/tf_saved_model'
TFLITE_MODEL_PATH = 'models/focus1/yolo11s_int8.tflite'

# 1. Load YOLO PT model and export to ONNX
model = YOLO(PT_MODEL_PATH)
model.export(format='onnx', imgsz=1024)  # By default saves as <modelname>.onnx

# Move or rename exported ONNX to desired path if necessary
if not os.path.exists(ONNX_MODEL_PATH):
    os.rename('Focus1_YOLO11s_x1024_14112024.onnx', ONNX_MODEL_PATH)

# 2. Convert ONNX to TensorFlow SavedModel using onnx2tf
convert(
    input_onnx_file_path=ONNX_MODEL_PATH,
    output_folder_path=TF_SAVED_MODEL_DIR,
    output_signaturedefs=True,          # Optional but recommended
    copy_onnx_input_output_names_to_tflite=True
)

# 3. Define a representative dataset generator for INT8 quantization
def representative_dataset():
    # Replace with your actual calibration images or dataset loader
    for _ in range(100):
        # Example: yield random tensor of input shape (1, 1024, 1024, 3), float32 [0,1]
        yield [tf.random.uniform(shape=(1, 1024, 1024, 3), dtype=tf.float32)]

# 4. Convert TensorFlow SavedModel to INT8 quantized TFLite model
converter = tf.lite.TFLiteConverter.from_saved_model(TF_SAVED_MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8   # or tf.int8 depending on model requirement
converter.inference_output_type = tf.uint8  # or tf.int8

tflite_model = converter.convert()

# Save the quantized TFLite model
with open(TFLITE_MODEL_PATH, 'wb') as f:
    f.write(tflite_model)

print(f'TFLite INT8 model saved to: {TFLITE_MODEL_PATH}')


ModuleNotFoundError: No module named 'tensorflow.python.distribute.distribution_strategy_context'

# Export NCNN

In [6]:
from ultralytics import YOLO

# Load a YOLO11 PyTorch model
model = YOLO(r"..\models\original\Focus1_YOLO11x_x1024_14112024.pt")

# Export the model to NCNN format
model.export(format="ncnn", imgsz = 640, half = True)  # creates 'yolo11n_ncnn_model'

# # Load the exported NCNN model
# ncnn_model = YOLO("yolo11n_ncnn_model")

Ultralytics 8.3.177  Python-3.11.13 torch-2.8.0+cu128 CPU (AMD Ryzen 7 4800H with Radeon Graphics)
YOLO11x summary (fused): 190 layers, 56,829,334 parameters, 0 gradients, 194.4 GFLOPs

PyTorch: starting from 'C:\Users\ADMIN\Desktop\Work\Master\ReUsePython\weights\..\models\original\Focus1_YOLO11x_x1024_14112024.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (109.2 MB)

TorchScript: starting export with torch 2.8.0+cu128...
TorchScript: export success  10.3s, saved as 'C:\Users\ADMIN\Desktop\Work\Master\ReUsePython\weights\..\models\original\Focus1_YOLO11x_x1024_14112024.torchscript' (217.9 MB)

NCNN: starting export with NCNN 1.0.20250503...
NCNN: running 'C:\Users\ADMIN\Desktop\Work\Master\ReUsePython\.venv\Lib\site-packages\ultralytics\pnnx.exe C:\Users\ADMIN\Desktop\Work\Master\ReUsePython\weights\..\models\original\Focus1_YOLO11x_x1024_14112024.torchscript ncnnparam=C:\Users\ADMIN\Desktop\Work\Master\ReUsePython\weights\..\models\original\Focus1_YOLO11

'C:\\Users\\ADMIN\\Desktop\\Work\\Master\\ReUsePython\\weights\\..\\models\\original\\Focus1_YOLO11x_x1024_14112024_ncnn_model'